<a href="https://colab.research.google.com/github/sominshim/Predicting_Personality_through_Text/blob/Ujin/preprocessing%26prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 불러오기

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import files
myfile = files.upload()

Saving data_winter_final.csv to data_winter_final.csv


In [25]:
data = pd.read_csv("data_winter_final.csv")
data.head()

,type,posts
0,ESTP,1. 최근 카페의 질의응답 게시판에 지속적으로 등업을 요구하는 글이 올라오고 있습니...
1,INFP,"제게 큰 현타를 주었던 글이에요.제 자신이 마음에 안 들고,늘 부족한 것 같고,절대..."
2,ISFJ,"유고슬라비아는 남슬라브 국가들(세르비아, 슬로베니아, 크로아티아, 보스니아, 북마케..."
3,ISTP,10대 여자고 키도 크고 나이에 비해 성숙한 이미지를 가지고 있어요. 잇팁답게 차갑...
4,ISFP,제가 요즘 끌리는 남자가있는데 esfp인거같아요esfj인줄 알았는데 상황에따라 행동...


# post 전처리

- 길이 50이상

In [35]:
data = data[data.posts.str.len()>50]
data = data.reset_index(drop=True)
data

,type,posts
0,ESTP,1. 최근 카페의 질의응답 게시판에 지속적으로 등업을 요구하는 글이 올라오고 있습니...
1,INFP,"제게 큰 현타를 주었던 글이에요.제 자신이 마음에 안 들고,늘 부족한 것 같고,절대..."
2,ISFJ,"유고슬라비아는 남슬라브 국가들(세르비아, 슬로베니아, 크로아티아, 보스니아, 북마케..."
3,ISTP,10대 여자고 키도 크고 나이에 비해 성숙한 이미지를 가지고 있어요. 잇팁답게 차갑...
4,ISFP,제가 요즘 끌리는 남자가있는데 esfp인거같아요esfj인줄 알았는데 상황에따라 행동...
...,...,...
20747,INFJ,나는 이 카페서 글보고 덧글달고 글쓰는 반면옆의 친구(ISTJ)는 축구보고 뉴스보고...
20748,ENFJ,10년넘은 친구가 있는데 그친구의 성격으로 제가 추측해봤을 때 그 친구는 E*FP였...
20749,INFJ,"구성지음, 갈라놓음. istj가 아닐까요.어느날 제가 목이 말라서 친구녀석에게 물좀..."
20750,ISTJ,이런 유형의 비슷한 질문이 있기는 했지만 대부분 ISTJ 남자와 잘맞을 거 같은 여...


### 0. Basic

- 가장 기초적인 전처리
- html tag 제거
- 숫자 제거
- Lowercasing
- "@%*=()/+ 와 같은 punctuation 제거/

In [36]:
posts = data['posts']
posts

0        1. 최근 카페의 질의응답 게시판에 지속적으로 등업을 요구하는 글이 올라오고 있습니...
1        제게 큰 현타를 주었던 글이에요.제 자신이 마음에 안 들고,늘 부족한 것 같고,절대...
2        유고슬라비아는 남슬라브 국가들(세르비아, 슬로베니아, 크로아티아, 보스니아, 북마케...
3        10대 여자고 키도 크고 나이에 비해 성숙한 이미지를 가지고 있어요. 잇팁답게 차갑...
4        제가 요즘 끌리는 남자가있는데 esfp인거같아요esfj인줄 알았는데 상황에따라 행동...
                               ...                        
20747    나는 이 카페서 글보고 덧글달고 글쓰는 반면옆의 친구(ISTJ)는 축구보고 뉴스보고...
20748    10년넘은 친구가 있는데 그친구의 성격으로 제가 추측해봤을 때 그 친구는 E*FP였...
20749    구성지음, 갈라놓음. istj가 아닐까요.어느날 제가 목이 말라서 친구녀석에게 물좀...
20750    이런 유형의 비슷한 질문이 있기는 했지만 대부분 ISTJ 남자와 잘맞을 거 같은 여...
20751    학교 말듣 숙제로 백상예술대상을 조사해 가야 하는데 어떻게 해야 할지 잘 모르겠어요...
Name: posts, Length: 20752, dtype: object

 한국어 문장 분리 : kss 라이브러리사용

In [9]:
!pip install kss

In [40]:
import kss

In [10]:
import re

def clean_text(texts):
    corpus = []
    for i in range(0, len(texts)):
        review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"]', '',str(texts[i])) #remove punctuation
        review = re.sub(r'\d+','', str(texts[i]))# remove number
        review = review.lower() #lower case
        review = re.sub(r'\s+', ' ', review) #remove extra space
        review = re.sub(r'<[^>]+>','',review) #remove Html tags
        review = re.sub(r'\s+', ' ', review) #remove spaces
        review = re.sub(r"^\s+", '', review) #remove space from start
        review = re.sub(r'\s+$', '', review) #remove space from the end
        review = re.sub(r'(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', '', review) # remove url
        #review = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', '', review) 
        
        #remove email
        review = re.sub(r'([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)', '_Email_', review)
        # 홈페이지 주소 제거
        review = re.sub(r'www.\S+','_homepage_', review)

        # 특수기호제거
        review = re.sub(r'[^\w\s]', '', review) #문제점 :  .마침표, ''인용구 등이 구분되지 않는데 괜찮은건가?
   
        # 특수기호제거
        review = re.sub(r'[▶■•]', '', review)

        #mbti 제거
        review = re.sub('infj|entp|intp|intj|entj|enfj|infp|enfp|isfp|istp|isfj|istj|estp|esfp|estj|esfj',"", review)

        corpus.append(review)
    return corpus

## 2. Spell check

- 띄어쓰기 검사

In [11]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-d80b9fzy
  Running command git clone -q https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-d80b9fzy
  Created wheel for pykospacing: filename=pykospacing-0.4-cp36-none-any.whl size=2255638 sha256=51a05d7112a71c304b17461422fcadcff27b1e88a73236e24dd9e4c169f7c8e2
  Stored in directory: /tmp/pip-ephem-wheel-cache-z7peskca/wheels/4d/45/58/e26cb2b7f6a063d234158c6fd1e5700f6e15b99d67154340ba
Successfully built pykospacing


In [12]:
from pykospacing import spacing 

- 맞춤법 검사기

In [13]:
!pip install git+https://github.com/ssut/py-hanspell.git

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-70k61wex
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-70k61wex
  Created wheel for py-hanspell: filename=py_hanspell-1.1-cp36-none-any.whl size=4854 sha256=10c0d4342037ad6f14c5701eb7dea12d0776c8b8f6fc81f4a7d281389ab3eaed
  Stored in directory: /tmp/pip-ephem-wheel-cache-b8obngvl/wheels/0a/25/d1/e5e96476dbb1c318cc26c992dd493394fe42b0c204b3e65588
Successfully built py-hanspell


In [14]:
from hanspell import spell_checker

- 반복되는 이모티콘이나 자모를 normalization

In [15]:
!pip install soynlp

     |████████████████████████████████| 419kB 6.3MB/s 


In [16]:
from soynlp.normalizer import *

- 외래어 사전

In [17]:
!curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=1RNYpLE-xbMCGtiEHIoNsCmfcyJP3kLYn" > /dev/null
!curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=1RNYpLE-xbMCGtiEHIoNsCmfcyJP3kLYn" -o confused_loanwords.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0    836      0 --:--:-- --:--:-- --:--:--   834
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 19779  100 19779    0     0  28832      0 --:--:-- --:--:-- --:--:-- 28832


In [18]:
lownword_map = {}
lownword_data = open('/content/confused_loanwords.txt', 'r', encoding='utf-8')

lines = lownword_data.readlines()

for line in lines:
    line = line.strip()
    miss_spell = line.split('\t')[0]
    ori_word = line.split('\t')[1]
    lownword_map[miss_spell] = ori_word

#### spell_check_text 함수화

In [19]:
def spell_check_text(texts):
    corpus = []
    for sent in texts:
        spaced_text = spacing(sent)
        spelled_sent = spell_checker.check(sent)
        checked_sent = spelled_sent.checked
        normalized_sent = repeat_normalize(checked_sent)
        for lownword in lownword_map:
            normalized_sent = normalized_sent.replace(lownword, lownword_map[lownword])
        corpus.append(normalized_sent)
    return corpus

## 3. part-of Speech

Python 기반의 형태소 분석기 중, 성능이 가장 좋은 것 중 하나인 카카오의 [Khaiii](https://github.com/kakao/khaiii)를 사용하겠습니다.

In [20]:
!git clone https://github.com/kakao/khaiii.git
!pip install cmake
!mkdir build
!cd build && cmake /content/khaiii
!cd /content/build/ && make all
!cd /content/build/ && make resource
!cd /content/build && make install
!cd /content/build && make package_python
!pip install /content/build/package_python

Cloning into 'khaiii'...
remote: Enumerating objects: 123, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (97/97), done.
remote: Total 1000 (delta 38), reused 61 (delta 22), pack-reused 877
Receiving objects: 100% (1000/1000), 33.06 MiB | 26.08 MiB/s, done.
Resolving deltas: 100% (396/396), done.
-- [hunter] Initializing Hunter workspace (70287b1ffa810ee4e952052a9adff9b4856d0d54)
-- [hunter]   https://github.com/ruslo/hunter/archive/v0.23.34.tar.gz
-- [hunter]   -> /root/.hunter/_Base/Download/Hunter/0.23.34/70287b1
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- w

In [42]:
from khaiii import KhaiiiApi
api = KhaiiiApi()

In [22]:
significant_tags = ['NNG', 'NNP', 'NNB', 'VV', 'VA', 'VX', 'MAG', 'MAJ', 'XSV', 'XSA'] # significant_tags -> 필요한 단어만 선택 , 불필요한 조사나 수사 같은 단어를 제거해주기 위해
# NNG : 일반 명사 / 'NNP' : 고유명사 / 'NNB' : 의존명사 / 'VV' : 동사 / 'VA' : 형용사 / 'VX' : 보조용언 / 'MAG' : 일반 부사 / 'MAJ' : 접속 부사 / 'XSV' : 동사파생접미사 / 'XSA' : 형용사 파생 접미사
def pos_text(texts):
    corpus = []
    for sent in texts:
        if sent=='':
          continue
        pos_tagged = ''
        for word in api.analyze(sent):
            for morph in word.morphs:
                if morph.tag in significant_tags:
                    pos_tagged += morph.lex + '/' + morph.tag + ' '
        corpus.append(pos_tagged.strip())
    return corpus


## 4. Stemming

동사를 원형으로 복원하도록 하겠습니다.
규칙은 다음과 같습니다.

1. NNG|NNP|NNB + XSV|XSA --> NNG|NNP|NNB + XSV|XSA + 다
2. NNG|NNP|NNB + XSA + VX --> NNG|NNP + XSA + 다
3. VV --> VV + 다
4. VX --> VX + 다

In [23]:
p1 = re.compile('[가-힣A-Za-z0-9]+/NN. [가-힣A-Za-z0-9]+/XS.')
p2 = re.compile('[가-힣A-Za-z0-9]+/NN. [가-힣A-Za-z0-9]+/XSA [가-힣A-Za-z0-9]+/VX')
p3 = re.compile('[가-힣A-Za-z0-9]+/VV')
p4 = re.compile('[가-힣A-Za-z0-9]+/VX')

In [24]:
def stemming_text(text):
    corpus = []
    for sent in text:
        ori_sent = sent
        mached_terms = re.findall(p1, ori_sent)
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '):
                lemma = term.split('/')[0]
                tag = term.split('/')[-1]
                modi_terms += lemma
            modi_terms += '다/VV'
            ori_sent = ori_sent.replace(ori_terms, modi_terms)
        
        mached_terms = re.findall(p2, ori_sent)
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '):
                lemma = term.split('/')[0]
                tag = term.split('/')[-1]
                if tag != 'VX':
                    modi_terms += lemma
            modi_terms += '다/VV'
            ori_sent = ori_sent.replace(ori_terms, modi_terms)

        mached_terms = re.findall(p3, ori_sent)
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '):
                lemma = term.split('/')[0]
                tag = term.split('/')[-1]
                modi_terms += lemma
            if '다' != modi_terms[-1]:
                modi_terms += '다'
            modi_terms += '/VV'
            ori_sent = ori_sent.replace(ori_terms, modi_terms)

        mached_terms = re.findall(p4, ori_sent)
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '):
                lemma = term.split('/')[0]
                tag = term.split('/')[-1]
                modi_terms += lemma
            if '다' != modi_terms[-1]:
                modi_terms += '다'
            modi_terms += '/VV'
            ori_sent = ori_sent.replace(ori_terms, modi_terms)
        corpus.append(ori_sent)
    return corpus

## 5. Stopwords

불용어는 도메인에 맞춰서 다양하게 구축될 수 있습니다.   

In [37]:
# NNG : 일반 명사 / 'NNP' : 고유명사 / 'NNB' : 의존명사 / 'VV' : 동사 / 'VA' : 형용사 / 'VX' : 보조용언 / 'MAG' : 일반 부사 / 'MAJ' : 접속 부사 / 'XSV' : 동사파생접미사 / 'XSA' : 형용사 파생 접미사
stopwords = ['데/NNB', '좀/MAG', '수/NNB', '등/NNB']

In [38]:
def remove_stopword_text(text):
    corpus = []
    for sent in text:
        modi_sent = []
        for word in sent.split(' '):
            if word not in stopwords:
                modi_sent.append(word)
        corpus.append(' '.join(modi_sent))
    return corpus

# 데이터에 적용

In [ ]:
new_posts = []
i=0
for post in posts:
  print(i)
  post_n = kss.split_sentences(post)

  #cleaned_corpus = []
  #for sent in post_n:
  #    cleaned_corpus.append(clean_punc(sent, punct, punct_mapping))

  post_n = clean_text(post_n)
  spell_preprocessed_corpus = spell_check_text(post_n)
  pos_tagged_corpus = pos_text(spell_preprocessed_corpus)
  stemming_corpus = stemming_text(pos_tagged_corpus)
  removed_stopword_corpus = remove_stopword_text(stemming_corpus)
  removed_stopword_corpus = ' '.join(removed_stopword_corpus)
  removed_stopword_corpus = removed_stopword_corpus.replace('/', '')
  removed_stopword_corpus = re.sub('[a-zA-Z]' , '', removed_stopword_corpus)
  new_posts.append(removed_stopword_corpus)
  i=i+1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [45]:
new_posts

['최근 카페 질의응답 게시판 지속 등업 요구하다 글 올라오다 있다 엠헬 가입인사 코너 개 댓글 달다 별도 요청 글 없이 자동 등업 되다 시스템 규정 읽다 않다 올라오다 조건 등업 요청 글 대하다 스태프 별도 안내 드리다 않다 규정 숙지하다 즐겁 엠 헬 활동하다 바라다 신고 게시글 잇따르다 동성애 및 종교 관련하다 내용 올라오다 있다 종교 및 성지 향상 대하다 개인 신념 모두 존중 받다 하 심리 카페인 엠헬 대하다 논쟁 지나치 깊 이루어지다 것 카페 취지 어긋나다 카페 내 질서 평화 위하다 앞 자신 인 소개용하다 언급되다 것 논쟁 위하다 종교 성지향 글 발견되다 무통보 삭제하다 하다 읽다 주다 감사하다',
 '크 연타 주다 글 자신 마음 안 들다 늘 부족하다 것 같 절대 사랑하다 없 사람 글 읽다 무릎 치다 대 후반 진짜 모르다 때 주변 주다 평가 사람 규정하다 버리다 거 이기 못 되 날 실수 하다 동생 못 하 애 것 하다 진짜 말 부모 주변 사람 잘 모르다 하다 말 않다 그리고 그때 아이 배우다 가다 하다 때 가혹하다 잣대 말 격려 인정 말 더 필요하다 그래서 삶 중간 진짜 돌아보다 시간 있다 하다 사람 평가 시각 둘러싸이다 번데기 뚫다 나오다 진짜 날개 바라보다 시각 갖추다 하다 제대로 바라보다 길 찾아가다 있다 상담실 처음 오다 때 기력하다 지치다 안정하다 모습 상담 통하다 마음 가볍 지다 방향 키 확실히 잡다 일상 파도 중심 잡다 자신 있다 나아가다 히말라야 산 등반 꼭 필요하다 계파 마음 여행 전문가 상담사하다 동행자 함께 하다 상담 안내 대 상담 온라인 줌 상담 모두 가능하다 유료 상담 회기당 분 달 끝나다 장기 상담회 기 이상 진행되다 불안 우울감 기력 대인 관계 문제 가족 문제 이별 번 아웃 주제 나눕다 감정 욕구 감각 알아차리다 내면 미 해결 과제 치유하다 게슈탈트 심리 상담 선호 유형 현재 심리 상태 탐색하다 활용하다 출처 수석 매니저 카 가님 블로그 개인 랜딩 페이지 소개 카페 내 강사 프로필 소개 글 유료 상담 신청 및 문의 카카오 채널 

In [ ]:
new_posts

In [ ]:
data['clean_post'] = new_posts

## 벡터화 : TfidfVectorizer

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfv = TfidfVectorizer().fit(new_posts)
print(tfidfv.transform(new_posts).toarray())
print(tfidfv.vocabulary_)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.0692911  0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.28898943 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
{'최근': 1444, '카페': 1479, '질의응답': 1398, '게시판': 83, '지속': 1370, '등업': 429, '요구하다': 1065, '올라오다': 1048, '있다': 1211, '엠헬': 989, '가입인사': 19, '코너': 1490, '댓글': 367, '달다': 340, '별도': 636, '요청': 1070, '없이': 976, '자동': 1219, '되다': 404, '시스템': 843, '규정': 181, '읽다': 1205, '않다': 910, '조건': 1309, '대하다': 364, '스태프': 827, '안내': 905, '드리다': 415, '숙지하다': 816, '즐겁': 1357, '활동하다': 1660, '바라다': 587, '신고': 850, '게시글': 82, '잇따르다': 1208, '동성애': 398, '종교': 1321, '관련하다': 145, '내용': 281, '성지': 779, '향상': 1625, '개인': 56, '신념': 853, '모두': 530, '존중': 1318, '받다': 602, '심리': 865, 

# Type

In [ ]:
# Split mbti personality into 4 letters and binarize
titles = ["Extraversion (E) - Introversion (I)",
          "Sensation (S) - INtuition (N)",
          "Thinking (T) - Feeling (F)",
          "Judgement (J) - Perception (P)"
         ] 
b_Pers = {'I':0, 'E':1, 'N':0, 'S':1, 'F':0, 'T':1, 'J':0, 'P':1}
b_Pers_list = [{0:'I', 1:'E'}, {0:'N', 1:'S'}, {0:'F', 1:'T'}, {0:'J', 1:'P'}]


#transform mbti to binary vector
def translate_personality(personality):
    return [b_Pers[l] for l in personality]

#transform binary vector to mbti personality
def translate_back(personality):
    s = ""
    for i, l in enumerate(personality):
        s += b_Pers_list[i][l]
    return s

list_personality_bin = np.array([translate_personality(p) for p in data.type])
print("Binarize MBTI list: \n%s" % list_personality_bin)

In [ ]:
data['I-E'] = list_personality_bin[:,0]
data['N-S'] = list_personality_bin[:,1]
data['F-T'] = list_personality_bin[:,2]
data['J-P'] = list_personality_bin[:,3]

# train / test set

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(data, data[['type']]):
    strat_train_set = data.loc[train_index]
    strat_test_set = data.loc[test_index]

X_train = strat_train_set.clean_post
X_test = strat_test_set.clean_post
#y_train = list_personality_bin[train_index]
#y_test = list_personality_bin[test_index]

print(X_train.shape)
print(X_test.shape)

In [ ]:
NB_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(max_features=100)),
                ('clf', OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
            ])
for category in categories:
    print('... Processing {}'.format(category))
    # X_train_over, y_train_over = smote.fit_sample(X_train, train[category])
    NB_pipeline.fit(X_train, train[category])
    prediction = NB_pipeline.predict(X_test)
    print('Test accuracy is {}'.format(accuracy_score(test[category], prediction)))
    print(confusion_matrix(test[category], prediction))
    # print(f1_score(test[category], prediction))
    print(classification_report(test[category], prediction, target_names=['0', '1']))

    from sklearn.metrics import roc_curve
    # test[category], prediction /X_train, train[category]
    fpr, tpr, thresholds = roc_curve(train[category], NB_pipeline.predict_proba(X_train)[:, 1])
    plt.plot(fpr, tpr, 'o-', label="Logistic Regression")
    plt.plot([0, 1], [0, 1], 'k--', label="random guess")
    # plt.plot([fallout], [recall], 'ro', ms=10)
    plt.xlabel('위양성률(Fall-Out)')
    plt.ylabel('재현률(Recall)')
    plt.title('Receiver operating characteristic example')
    plt.show()